# fMRI example

In [1]:
import numpy as np
from scipy.io import loadmat
from os.path import join
import glob
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
def center_normalize(x):
    return (x - np.mean(x)) / np.std(x)


In [3]:
#data dir test 
data_dir = '../data/greco_mri'
behav_dir = '../data/greco_behav'

#rois
rois = ['left_CA1','right_CA1','left_DG','right_DG'];

#subjec list
subjects = ['S1_A','S2_B','S3_A','S4_A','S5_A','S6_A','S7_A','S8_B',
            'S9_A','S10_B','S11_B','S12_B','S13_B','S14_B','S15_B',
            'S16_A','S21_B','S22_B','S23_B','S24_A'];

voxel_cutoff = 130;

subjects = np.array(subjects);


# does tf want a list or tuple?
data_y = [];
data_x = [];
data_y_TEST = [];
data_x_TEST = [];

sub_rand_idx = np.random.permutation(20);
subjects_train = subjects[sub_rand_idx[0:-1]]
subjects_test  = [subjects[sub_rand_idx[-1]]];

num_voxels = np.empty(subjects_train.shape)
num_trials = np.empty(subjects_train.shape)


for i,subj in enumerate(subjects_train):
    fname = join(data_dir,subj + '_right_DG.csv')
    betas= pd.read_csv(fname,header=None)
    behav = pd.read_table(join(behav_dir,subj,subj + '.txt'))
#     idx = np.random.permutation(100);
    idx = np.array(range(0,voxel_cutoff))

    for j in range(0,betas.shape[0]):
 
        data_x.append(center_normalize(betas.iloc[j,idx].values))
        data_y.append(behav['currCity'][j])
    num_trials[i] = betas.shape[0];
    num_voxels[i] = betas.shape[1];
 
for i,subj in enumerate(subjects_test):
    fname = join(data_dir,subj + '_right_DG.csv')
    betas= pd.read_csv(fname,header=None)
    behav = pd.read_table(join(behav_dir,subj,subj + '.txt'))
    idx = np.array(range(0,voxel_cutoff))

    for j in range(0,betas.shape[0]):

        data_x_TEST.append(center_normalize(betas.iloc[j,idx].values))
        data_y_TEST.append(behav['currCity'][j])

 
    

In [4]:
print('minimum number of voxels: ' + str(np.min(num_voxels)))
print('number of trials per subject: ' + str(np.mean(num_trials)))
print('number of subjects: ' + str(len(subjects_train)))

minimum number of voxels: 142.0
number of trials per subject: 99.9473684211
number of subjects: 19


In [5]:
# one hot encoding
ynp=np.array(data_y)
data_y_onehot=[0]*len(ynp)
for i,j in enumerate(ynp):
    data_y_onehot[i]=[0]*ynp.max()
    data_y_onehot[i][j-1]=1
    
    
# one hot encoding
ynp=np.array(data_y_TEST)
data_y_TEST_onehot=[0]*len(ynp)
for i,j in enumerate(ynp):
    data_y_TEST_onehot[i]=[0]*ynp.max()
    data_y_TEST_onehot[i][j-1]=1

In [6]:
# X_train, X_valid, Y_train, Y_valid = train_test_split(data_x, data_y_onehot, 
#                                                     test_size=0.2, 
#                                                     stratify=data_y_onehot)

X_train = data_x;
Y_train = data_y_onehot;

X_valid = data_x_TEST;
Y_valid = data_y_TEST_onehot;


In [7]:
import tensorflow as tf 

sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, shape=[None, voxel_cutoff])
y_ = tf.placeholder(tf.float32, shape=[None, 3])


W = tf.Variable(tf.zeros([voxel_cutoff,3]))
b = tf.Variable(tf.zeros([3]))

sess.run(tf.global_variables_initializer())

y = tf.matmul(x,W) + b

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [8]:
%%time
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)


for i in range(1000):
    train_step.run(feed_dict={x: X_train, y_: Y_train})


CPU times: user 5.31 s, sys: 251 ms, total: 5.56 s
Wall time: 4.14 s


In [9]:
#tf.argmax gives an index of the highest entry in a tensor along some axis
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

#we can take this list of booleans and calculate the fraction correct
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#print the accuracy
print(accuracy.eval(feed_dict={x: X_valid, y_: Y_valid}))

0.32


## Now let's build a multilayer convolutional network
  

In [48]:


def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    IMPORTANT: yhat is not softmax since TensorFlow's softmax_cross_entropy_with_logits() does that internally.
    """
    h    = tf.nn.sigmoid(tf.matmul(X, w_1))  # The \sigma function
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat


# Layer's sizes
x_size = voxel_cutoff   
h_size = 256               
y_size = 3  

# Symbols
X = tf.placeholder("float", shape=[None, x_size])
y = tf.placeholder("float", shape=[None, y_size])

# Weight initializations
w_1 = init_weights((x_size, h_size))
w_2 = init_weights((h_size, y_size))

# Forward propagation
yhat    = forwardprop(X, w_1, w_2)
predict = tf.argmax(yhat, axis=1)

# Backward propagation
cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
updates = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

# Run SGD
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for epoch in range(100):
    # Train with each example
    for i in range(len(X_train)):
        sess.run(updates, feed_dict={X: X_train[i: i + 1], y: Y_train[i: i + 1]})

    train_accuracy = np.mean(np.argmax(Y_train, axis=1) ==
                             sess.run(predict, feed_dict={X: X_train, y: Y_train}))
    test_accuracy  = np.mean(np.argmax(Y_valid, axis=1) ==
                             sess.run(predict, feed_dict={X: X_valid, y: Y_valid}))

    print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%"
          % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))

Epoch = 1, train accuracy = 37.60%, test accuracy = 35.00%
Epoch = 2, train accuracy = 39.44%, test accuracy = 33.00%
Epoch = 3, train accuracy = 39.86%, test accuracy = 33.00%
Epoch = 4, train accuracy = 40.55%, test accuracy = 30.00%
Epoch = 5, train accuracy = 41.34%, test accuracy = 31.00%
Epoch = 6, train accuracy = 42.02%, test accuracy = 30.00%
Epoch = 7, train accuracy = 42.92%, test accuracy = 30.00%
Epoch = 8, train accuracy = 43.44%, test accuracy = 32.00%
Epoch = 9, train accuracy = 44.39%, test accuracy = 30.00%
Epoch = 10, train accuracy = 45.44%, test accuracy = 29.00%
Epoch = 11, train accuracy = 46.55%, test accuracy = 30.00%
Epoch = 12, train accuracy = 48.29%, test accuracy = 30.00%
Epoch = 13, train accuracy = 49.97%, test accuracy = 29.00%
Epoch = 14, train accuracy = 51.45%, test accuracy = 28.00%
Epoch = 15, train accuracy = 54.13%, test accuracy = 30.00%
Epoch = 16, train accuracy = 56.40%, test accuracy = 30.00%
Epoch = 17, train accuracy = 59.35%, test accurac